In [4]:
!pip install transformers gradio requests


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import requests
import gradio as gr
import torch

# Initialize DialoGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Set up your NewsAPI key (replace 'YOUR_NEWSAPI_KEY' with an actual API key)
NEWS_API_KEY = "YOUR_NEWSAPI_KEY"
NEWS_API_URL = "https://newsapi.org/v2/top-headlines"

def fetch_news(category="general"):
    """
    Fetches top headlines from NewsAPI based on the specified category.
    """
    try:
        params = {
            "apiKey": "3ab040d360ba4403ae19d4c2ae2b8550",
            "category": category,
            "country": "us",  # You can change this to your preferred country code
            "pageSize": 5  # Number of headlines to fetch
        }
        response = requests.get(NEWS_API_URL, params=params)
        data = response.json()
        if data["status"] == "ok":
            headlines = [article["title"] for article in data["articles"]]
            return "\n".join([f"{idx + 1}. {headline}" for idx, headline in enumerate(headlines)])
        else:
            return "Unable to fetch news at the moment."
    except Exception as e:
        return f"Error fetching news: {str(e)}"

# Chatbot function for conversation and news handling
chat_history_ids = None

def news_nerd_bot(input_text):
    """
    Handles user inputs and provides responses for news queries or general chat.
    """
    global chat_history_ids

    # Check for news-related keywords
    if "news" in input_text.lower():
        if "technology" in input_text.lower():
            return fetch_news("technology")
        elif "sports" in input_text.lower():
            return fetch_news("sports")
        elif "health" in input_text.lower():
            return fetch_news("health")
        elif "business" in input_text.lower():
            return fetch_news("business")
        else:
            return fetch_news("general")

    # Handle general conversation using DialoGPT
    new_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors="pt")
    bot_input_ids = (
        torch.cat([chat_history_ids, new_input_ids], dim=-1)
        if chat_history_ids is not None
        else new_input_ids
    )

    # Generate response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Create Gradio interface
interface = gr.Interface(
    fn=news_nerd_bot,
    inputs="text",
    outputs="text",
    title="News Nerd AI Chatbot",
    description="A chatbot that provides news updates and engages in casual conversation.",
)

# Launch the chatbot in Colab
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://96f6740730c4f6dc8a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
